In [1]:
import pathlib
import os

import pandas as pd

import matplotlib.pyplot as plt
import datetime
%matplotlib inline

import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import mapclassify

In [2]:
import pymysql

In [3]:
db = pymysql.connect(host='localhost',
                     user='root',
                     password='password',
                     database='chicago')
cursor = db.cursor()

In [5]:
sqlcom ="SELECT * FROM weekday_morningPeak"
df= pd.read_sql(sqlcom, con=db)
print(df)

/opt/anaconda3/envs/python/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


             node_id         lat         lon            timestamp value_hrf
0       001e0611441e   41.808594  -87.665048  2019/09/30 04:00:00     56.72
1       001e0610bc10   41.736314  -87.624179  2019/09/30 04:00:02     85.02
2       001e061146cb  41.9140942  -87.683022  2019/09/30 04:00:02     62.74
3       001e0610ba13   41.751238   -87.71299  2019/09/30 04:00:04     56.72
4       001e06113acb   41.839066  -87.665685  2019/09/30 04:00:04     56.72
...              ...         ...         ...                  ...       ...
106851  001e0610ba13   41.751238   -87.71299  2019/10/04 08:59:57     56.72
106852  001e061130f4   41.896157  -87.662391  2019/10/04 08:59:57     56.72
106853  001e061183f5   41.692703   -87.62102  2019/10/04 08:59:57     62.74
106854  001e06113ace    41.83107  -87.617298  2019/10/04 08:59:58     56.72
106855  001e0610bbe5   41.736495  -87.614529  2019/10/04 08:59:59     56.72

[106856 rows x 5 columns]


Calculate the average sound level of each node during weekdays morning peak hours

In [7]:
from decimal import Decimal
dict={}
for i in range(0,len(df)):
    key=df.iloc[i]['node_id']
    lat=df.iloc[i]['lat']
    lon=df.iloc[i]['lon']
    value=float(df.iloc[i]['value_hrf'])
    
    if key in dict:
        tempDict=dict.get(key)
        valueTemp=tempDict["value"]
        valueTemp.append(value)
        dict[key]=tempDict 
    else:
        dict[key]={"value":[value],"lon":lon,"lat":lat}

temp={}
for key in dict:
    tempDict=dict.get(key)
#     tempDict['average']=Decimal(np.mean(tempDict["value"])).quantize(Decimal('0.00'))
    tempDict['average']=np.mean(tempDict["value"])
    temp[key]=tempDict
print(temp)

{'001e0611441e': {'value': [56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 62.74, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 55.0, 62.74, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 55.0, 55.0, 56.72, 55.0, 55.0, 55.0, 56.72, 55.0, 56.72, 56.72, 56.72, 55.0, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 55.0, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 56.72, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 55.0, 55.0, 55.0, 56.72, 55.0, 56.72, 55.0, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 55.0, 55.0, 56.72, 56.72, 55.

In [8]:
df=pd.DataFrame.from_dict(temp,orient='index')
df

,value,lon,lat,average
001e0611441e,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.665048,41.808594,57.214022
001e0610bc10,"[85.02, 85.35, 85.35, 85.02, 85.67, 85.35, 85....",-87.624179,41.736314,86.774787
001e061146cb,"[62.74, 62.74, 62.74, 66.26, 62.74, 62.74, 62....",-87.683022,41.9140942,59.368924
001e0610ba13,"[56.72, 56.72, 55.0, 55.0, 55.0, 55.0, 55.0, 5...",-87.71299,41.751238,56.649477
001e06113acb,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.665685,41.839066,58.589022
001e06113d20,"[56.72, 56.72, 56.72, 55.0, 56.72, 55.0, 56.72...",-87.6116431,41.8920031,56.386759
001e061183f5,"[56.72, 62.74, 66.26, 62.74, 62.74, 62.74, 62....",-87.62102,41.692703,61.274622
001e06118509,"[62.74, 62.74, 62.74, 62.74, 62.74, 62.74, 62....",-87.654487,41.779744,64.168571
001e0610bbe5,"[55.0, 55.0, 55.0, 55.0, 56.72, 55.0, 55.0, 55...",-87.614529,41.736495,56.809465
001e0610ee43,"[56.72, 56.72, 56.72, 56.72, 56.72, 56.72, 56....",-87.598713,41.788608,58.199909


In [9]:
df.shape

(35, 4)

In [10]:
df['average']

001e0611441e    57.214022
001e0610bc10    86.774787
001e061146cb    59.368924
001e0610ba13    56.649477
001e06113acb    58.589022
001e06113d20    56.386759
001e061183f5    61.274622
001e06118509    64.168571
001e0610bbe5    56.809465
001e0610ee43    58.199909
001e06112e77    56.873571
001e06113107    57.185183
001e061130f4    57.509791
001e0610f703    59.846163
001e0610e538    61.590411
001e061182a7    62.047298
001e0610f05c    62.381616
001e061144be    57.297497
001e0610ba46    60.848065
001e06113ad8    57.484110
001e0610ee5d    58.490183
001e0610f732    61.951959
001e06113ace    56.906405
001e0610ba15    56.395117
001e06113a24    57.291870
001e061146ba    56.849559
001e06118295    60.826457
001e0611536c    57.293537
001e06117b44    62.519623
001e0610ee36    58.670584
001e0610f6db    57.054895
001e0611850f    62.131066
001e061184a3    62.938452
001e061144cd    56.820348
001e0611462f    56.408204
Name: average, dtype: float64